### Imports and Authentication

In [ ]:
import pandas as pd
from ddb_auth import DDBAuth #https://github.com/arup-group/ddbpy_auth
import ddb_get_post as pyddb
import arupcomputepy
import json

### Inputs

In [ ]:
# pyddb.site = "sandbox"
# job_number = "27846500"

%store -r env
%store -r job_number

pyddb.site = env

### Calculation Settings
Enter the DC2 calc_id, DDB parameter type ids, and DC2 variable names.

The names do not have to match the DDB parameter names.


Note: This is currently only set up for DC2 calcs with a single output.

In [ ]:
calc_id = 4374727

input_dict = {
    
    "Lighting heat gain" : {
        "id" : "9a427e74-d428-44d0-aa8a-1744e5cdbb3d",
        "dc2_var" : "Phi_lighting",
    },
              
    "Equipment heat gain" : {
        "id" : "36f829c4-c6f6-40bd-8421-d29930193a95",
        "dc2_var" : "Phi_equipment",
    },

    "Total fabric heat gain" : {
        "id" : "2f0fe574-d37c-4515-ad1b-dd1d1b06d8c6",
        "dc2_var" : "Phi_fabric",
    },
              
    "Total occupancy sensible heat gain" : {
        "id" : "24e0d62a-ca93-4325-a6c5-7704089099ac",
        "dc2_var" : "Phi_PeopleSensible",
    },
}


# sensible heat gain _ summer
output_parameter_id = "2770a38e-42d0-423e-9856-d6f452a9f965" # will update to an output dict when necessary
output_unit_id = "2a029d23-0fcf-4170-93f0-285960937099"

space_instance_id = '21ba5fbb-f079-4c37-a3b9-f36e21e5b3ac'

for parameter in input_dict:
    input_dict[parameter]["list"] = []

### Get Values from DDB

Get project_id from job_number.
Get all parameters, filtered by project, asset type, and the parameters we want

In [ ]:
project_id = pyddb.get_project_id(job_number)

space_instance_parameters = pyddb.get_parameters(
    project_id = project_id,
    asset_type = space_instance_id,
    parameter_type_id = [input_dict[key]["id"] for key in input_dict.keys()],
    page_limit = 9999
)

### Data Manipulation

Building a dictionary of parameter values and asset_ids

In [ ]:
spaces = list(set([x["parents"][0]["name"] for x in space_instance_parameters]))

# convert json to list of tuples containing parameter names, values, space names, space instance ids
data = [(x["parents"][0]["name"], x["parameter_type"]["id"], x["revision"]["values"][0]["value"]) for x in space_instance_parameters]

d = {space:{x[1]: x[2] for x in data if x[0] == space} for space in spaces}

for x in space_instance_parameters:
    d[x["parents"][0]["name"]]["id"] = x["parents"][0]["id"]

Create lists of values, skipping and displaying spaces where all values are not available.

In [ ]:
space_list = []

for space in spaces:
    try:
        values = []
        for parameter in input_dict:
            values.append(d[space][input_dict[parameter]["id"]])
        if len(values) == len(input_dict):
            for i, parameter in enumerate(input_dict):
                input_dict[parameter]["list"].append(values[i])
            space_list.append(d[space]["id"])
    except KeyError:
        print(f"Parameter '{parameter}' missing in {space}.")

### DesignCheck Batch Calculation
Defines a function to post batch calculations, then runs it with our inputs.

In [7]:
def designcheck_batchcalc(inputs):

    token = arupcomputepy.AcquireNewAccessTokenDeviceFlow()
    response = arupcomputepy.MakeCalculationRequest(
        inputs["calc_id"],
        inputs["job_number"],
        token,
        isBatch=True,
        variables=inputs["variables"],
    )
    results = json.loads(response["output"])
    output = []
    for result in results:
        if "errors" in result:
            for error in result["errors"]:
                print(error)
                raise Exception
        entry = []
        for value in result["arupComputeResultItems"]:

            entry.append(value["value"])

        output.append(entry)

    return output

inputs = {
    "calc_id": calc_id,
    "job_number": job_number,
    "variables": {
        "ID": space_list,
    },
}

for parameter in input_dict:
    inputs["variables"][input_dict[parameter]["dc2_var"]] = input_dict[parameter]["list"]

output_list = [x[0] for x in designcheck_batchcalc(inputs)]

KeyboardInterrupt: 

### Upload to DDB

Creates a source with the DesignCheck2 calc as a reference.
Posts all output values to their respective assets.

#### NOTE:
This post is not batched and can be significantly improved if it were.

In [ ]:
if output_list:

    source_id = pyddb.add_new_source(
        project_id = project_id,
        source_type_id = "3169ba8f-d474-4ee3-8891-09817a03b196",
        source_title = "DesignCheck2",
        source_reference = str(calc_id)
    )

    for i, space in enumerate(space_list):
        pyddb.add_update_parameter(
            parameter_type_id = output_parameter_id,
            project_id = project_id,
            asset_id = space,
            value = output_list[i],
            unit_id = output_unit_id,
            source_id = source_id)